In [1]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [3]:
import torch
import torch.nn as nn
from utils.attention import MultiHeadAttention
from utils.layers import FeedForward, LayerNorm

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=config["emb_dim"],
            d_out=config["emb_dim"],
            context_length=config["context_length"],
            dropout=config["drop_rate"],
            num_heads=config["n_heads"],
            qkv_bias=config["qkv_bias"]
        )
        self.ff = FeedForward(config)
        self.norm1 = LayerNorm(config["emb_dim"])
        self.norm2 = LayerNorm(config["emb_dim"])
        self.drop_skip_layer = nn.Dropout(config["drop_rate"])

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_skip_layer(x)
        x = x + shortcut
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_skip_layer(x)
        x = x + shortcut
        return x

In [12]:
class GPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"],cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"],cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])

        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self,in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len,device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits




In [13]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)


tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [14]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.4093,  0.6366, -0.3657,  ...,  0.2840,  0.2277, -0.4068],
         [ 0.1320, -0.5946, -0.8225,  ...,  0.1029,  0.3817, -0.5678],
         [ 1.1542, -0.1230,  0.0244,  ..., -0.0413, -0.6898, -0.1426],
         [-1.1784,  0.2778, -0.1429,  ...,  0.7957,  0.3100, -0.0979]],

        [[-0.1427, -0.0037, -0.3014,  ...,  0.7300,  0.4793, -0.8115],
         [-0.0651,  0.2773,  0.1059,  ...,  0.7891, -0.1348,  0.3851],
         [ 0.9832,  1.0264, -0.3386,  ...,  0.8984,  0.4766, -0.3605],
         [ 0.0496,  0.2019,  0.4266,  ...,  1.0026, -0.5675, -0.0563]]],
       grad_fn=<UnsafeViewBackward0>)
